In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# ! python -m pip install albumentations

In [3]:
# ! python -m pip install pretrainedmodels

In [4]:
import os
import helpers
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from dataset import BengaliTrainDataset
from models import ResNet34
from tqdm import tqdm

# Model Configuration 

In [5]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
TRAINING_FOLDS_CSV = "input/train_folds.csv"
EPOCHS = 10

TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE = 8

In [6]:
def loss_fn(outputs, targets):
    o1, o2, o3 = outputs
    t1, t2, t3 = targets
    l1 = nn.CrossEntropyLoss()(o1, t1)
    l2 = nn.CrossEntropyLoss()(o2, t2)
    l3 = nn.CrossEntropyLoss()(o3, t3)
    return (l1 + l2 + l3) / 3

def train(epoch, trainset, validset, train_loader, valid_loader, model, optimizer, device, itr):
    model.train()
    loss_hist = helpers.Averager()
    loss_hist.reset()
    for batch, dataset in tqdm(enumerate(train_loader), total=int(len(trainset)/train_loader.batch_size)):
        image = dataset["image"]
        grapheme_root = dataset["grapheme_root"]
        vowel_diacritic = dataset["vowel_diacritic"]
        consonant_diacritic = dataset["consonant_diacritic"]

        image = image.to(device, dtype = torch.float) # move data to cuda
        grapheme_root = grapheme_root.to(DEVICE, dtype = torch.long)
        vowel_diacritic = vowel_diacritic.to(DEVICE, dtype = torch.long)
        consonant_diacritic = consonant_diacritic.to(DEVICE, dtype = torch.long)

        outputs = model(image)
        targets = (grapheme_root, vowel_diacritic, consonant_diacritic)
        loss = loss_fn(outputs, targets)
        loss_hist.send(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        itr += 1
        
    model.eval()
    with torch.no_grad():
        final_loss = 0.
        for batch, dataset in tqdm(enumerate(valid_loader), total=int(len(validset)/valid_loader.batch_size)):
            image = dataset["image"]
            len(image)
            grapheme_root = dataset["grapheme_root"]
            vowel_diacritic = dataset["vowel_diacritic"]
            consonant_diacritic = dataset["consonant_diacritic"]

            image = image.to(DEVICE, dtype = torch.float) # move data to cuda
            grapheme_root = grapheme_root.to(DEVICE, dtype = torch.long)
            vowel_diacritic = vowel_diacritic.to(DEVICE, dtype = torch.long)
            consonant_diacritic = consonant_diacritic.to(DEVICE, dtype = torch.long)
            ## update the average validation loss
            outputs = model(image)
            targets = (grapheme_root, vowel_diacritic, consonant_diacritic)
            loss = loss_fn(outputs, targets)
            final_loss += loss
    nv = len(valid_loader)
    print(f"Epoch:{epoch} validation_loss: {final_loss / nv}")
        
    return final_loss / nv

def main(training_folds, validation_folds, model, device, train_batch_size, test_batch_size, epochs):
    model = model(pretrained = True)
    model.to('cuda')

    def train_transforms():
        return A.Compose([
            A.Resize(137, 236, always_apply = True),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=5, p=0.9),
            A.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225), always_apply = True)])

    def valid_transforms():
        return A.Compose([
            A.Resize(137, 236, always_apply = True),
            A.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225), always_apply = True)])
    
    train_df = pd.read_csv("input/train_folds.csv")
    valid_df = train_df[train_df["kfold"].isin(validation_folds)]
    train_dataset = BengaliTrainDataset(train_df, folds=training_folds, transforms = train_transforms())
    valid_dataset = BengaliTrainDataset(valid_df, folds=validation_folds, transforms = valid_transforms())
    print(len(train_dataset))
    print(len(valid_dataset))

    trainloader = DataLoader(
    dataset = train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=4
        )

    validloader = DataLoader(
    dataset = valid_dataset,
    batch_size=test_batch_size,
    shuffle=False,
    num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode = 'min', patience = 5, factor = 0.3)

    # if torch.cuda.device_count() > 1:
    #     model = nn.DataParallel(model)
    
    itr = 1
    for epoch in range(epochs):
        val_score = train(epoch,train_dataset,valid_dataset,trainloader,validloader,model,optimizer,DEVICE,itr)
        scheduler.step(val_score)
        torch.save(model.state_dict(), f"Models/resnet_34_fold{VALIDATION_FOLDS[0]}.bin")

# Training

In [7]:
TRAINING_FOLDS = [0,1,2,3]
VALIDATION_FOLDS = [4]
main(TRAINING_FOLDS, VALIDATION_FOLDS, ResNet34, DEVICE, TRAIN_BATCH_SIZE, TEST_BATCH_SIZE, EPOCHS)

(40168, 6)
160672
40168


2511it [05:36,  7.45it/s]                          
100%|██████████| 5021/5021 [03:54<00:00, 21.37it/s]


Epoch:0 validation_loss: 0.159388929605484


2511it [05:36,  7.46it/s]                          
100%|██████████| 5021/5021 [00:53<00:00, 94.36it/s]


Epoch:1 validation_loss: 0.12361336499452591


...

Epoch:7 validation_loss: 0.0974956676363945


2511it [05:36,  7.46it/s]                          
100%|██████████| 5021/5021 [00:53<00:00, 93.26it/s]


Epoch:8 validation_loss: 0.09610696136951447


2511it [05:36,  7.47it/s]                          
100%|██████████| 5021/5021 [00:53<00:00, 93.86it/s] 


Epoch:9 validation_loss: 0.09045566618442535


In [13]:
TRAINING_FOLDS = [0,1,2,4]
VALIDATION_FOLDS = [3]
main(TRAINING_FOLDS, VALIDATION_FOLDS, ResNet34, DEVICE, TRAIN_BATCH_SIZE, TEST_BATCH_SIZE, EPOCHS)

TypeError: main() missing 7 required positional arguments: 'training_folds', 'validation_folds', 'model', 'device', 'train_batch_size', 'test_batch_size', and 'epochs'

In [ ]:
TRAINING_FOLDS = "(0,1,4,3)"
VALIDATION_FOLDS = "(2,)"
main()

In [ ]:
TRAINING_FOLDS = "(0,4,2,3)"
VALIDATION_FOLDS = "(1,)"
main()

In [ ]:
TRAINING_FOLDS = "(4,1,2,3)"
VALIDATION_FOLDS = "(0,)"
main()